# 1) Introduction

L’objectif du notebook est de pré traiter les données issues de l’outil d’export de données de Stack Overflow-
"stackexchange explorer", qui recense un grand nombre de données authentiques de la plateforme afin de détecter
les sujets et générer des tags.

## 1.1. Étapes de prétraitement

**1) Suppression du bruit**

1. Suppression du formatage HTML
2. Suppression des contractions
3. La correction orthographique
4. Mettre en minuscule le texte

**2) Suppression des caractères simples**

1. Suppression de la ponctuation, des caractères spéciaux et des nombres
2. Suppression d'un seul caractère (facultatif et spécifique)

**3) Suppression de StopWords**

1. Suppression du mot le plus fréquent
2. Suppression d'un certain type de mot (facultatif et spécifique)

**4) Stemming / Lemmatisation**

1. Stemming
2. Lemmatisation


Ce pré-processus est utilisé pour effectuer une simple détection de sujet (LDA, NMF, etc.) ou une classification,
des informations nécessaires à certaines analyses peuvent être perdues.

 

1.1.1 Vocabulaire
Si vous êtes nouveau dans la PNL, voici une petite liste de concepts utilisés dans ce cahier.

Tokenize : "Processus de conversion d'une chaîne en une liste de sous-chaînes, appelées tokens."

Normalisation du texte : "Processus de transformation d'un texte en une seule forme canonique qu'il n'aurait
peut-être pas eu auparavant (par exemple, mettre en minuscule le texte, supprimer les contractions, correction
orthographique, stemming / lemmatisation, etc.). La normalisation du texte nécessite de savoir quel type de texte
doit être normalisée et comment elle doit être traitée par la suite ; il n’existe pas de procédure de
normalisation universelle. "

Suppression du bruit : "Processus de suppression de tout élément susceptible interferer avec votre analyse
(par exemple, suppression du code HTML, mettre en minuscule le texte, suppression de la ponctuation / du caractère
spécial, etc.)

Stemming: "Processus de réduction des mots à leur racine , base ou forme de racine - généralement une forme de
mot écrit ("fishing", "fished", and "fisher" to the stem "fish")."

Lemmatisation : "Processus de regroupement des formes fléchies d'un mot afin qu'elles puissent être analysées
comme un seul élément, identifié par le lemme du mot, ou par la forme du dictionnaire (ie : "walking" to "walk",
 "better" to "good")."

StopWord : "Mots qui sont filtrés avant ou après le traitement des données en langage naturel (texte). Les mots
d'arrêt font généralement référence aux mots les plus courants dans une langue (des mots comme "The","a", etc. en
anglais)."

# 2) Libraries and Dataset 

In [151]:
! pip install bs4
! pip install contractions
! pip install autocorrect 

In [152]:
# generic librairies
import pandas as pd

# Text librairies
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import contractions
from autocorrect import Speller

In [153]:
# https://numpy.org/devdocs/user/basics.types.html

dtypes_questions = {'Id':'int32', 'Score': 'int16', 'Title': 'string', 'Body': 'string', 'Tags': 'string'}

In [154]:
%%time

nrows=10000

df_questions_python = pd.read_csv('input/QueryResults python score more than 20.csv',
                           usecols=dtypes_questions.keys(),
                           encoding = "ISO-8859-1",
                           dtype=dtypes_questions,
                           nrows=nrows
                          )

df_questions_r = pd.read_csv('input/QueryResults r score more than 20.csv',
                           usecols=['Id', 'Score', 'Title', 'Body', 'Tags'],
                           encoding = "ISO-8859-1",
                           dtype=dtypes_questions,
                           nrows=nrows
                          )

df_questions = pd.concat([df_questions_python,df_questions_r])
df_questions['Title_raw']=df_questions['Title']
df_questions = df_questions.sample(frac=1).reset_index(drop=True)

CPU times: user 177 ms, sys: 16.1 ms, total: 193 ms
Wall time: 194 ms


In [155]:
df_questions[['Title', 'Body', 'Tags']] = df_questions[[
    'Title', 'Body','Tags'
]].applymap(lambda x: str(x).encode("utf-8", errors='surrogatepass').decode(
    "ISO-8859-1", errors='surrogatepass'))

In [156]:
df_questions.dtypes

Id            int32
Title        object
Body         object
Tags         object
Score         int16
Title_raw    string
dtype: object

In [157]:
# Remove all questions that have a < 30 score
df_questions = df_questions[df_questions["Score"] >= 20]

In [158]:
spell = Speller()
token = ToktokTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
charac = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'
stop_words = set(stopwords.words("english"))
adjective_tag_list = {'JJ', 'JJR', 'JJS', 'RBR', 'RBS'}  # List of Adjective's tag from nltk package


**Tag list**  
List of tag use in the tagger (pos_tag function) from NLTK:
https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [159]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13913 entries, 0 to 13912
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         13913 non-null  int32 
 1   Title      13913 non-null  object
 2   Body       13913 non-null  object
 3   Tags       13913 non-null  object
 4   Score      13913 non-null  int16 
 5   Title_raw  13913 non-null  string
dtypes: int16(1), int32(1), object(3), string(1)
memory usage: 625.0+ KB


In [160]:
df_questions['qTitle'] = df_questions['Tags'].apply(lambda x : len((x).split(" ")))

# 3) Suppression du bruit

La suppression du bruit consiste à supprimer tout ce qui peut interférer avec votre analyse de texte. C'est comme l'étape de nettoyage des données pour un projet ML classique.

## 3.1. Suppression du code HTML

In [161]:
df_questions['Body'][10]

'<p><strong>Question</strong>: How can I use <code>paste</code> without <code>100000</code> becoming <code>1e+05</code>?</p>\n\n<p>Sorry in advance if this question seems frivolous (but it has resulted in a bug in my code).  I use R to call an external script, so when I say e.g. <code>paste("abc",100000)</code> I want it to output <code>"abc 100000"</code> and not <code>"abc 1e+05"</code>.</p>\n\n<p>Here\'s an example of what it looks like on my screen:</p>\n\n<pre><code>&gt; paste("abc",100000)\n[1] "abc 1e+05"\n&gt; paste("abc",100001)\n[1] "abc 100001"\n</code></pre>\n\n<p>This results in the bizarre behaviour that my script works for the input "100001" but not "100000".</p>\n\n<p>I realise I could create a script to convert numbers to strings however I like, but I feel I shouldn\'t do this if there is an internal way to do the same thing (I suspect there is some "method" I\'m missing).</p>\n\n<p>[If it helps, I\'m on Ubuntu 12.04.1 LTS ("precise"), running R version 2.14.1 (2011-12

In [162]:
%%time

# Analyser la question et le titre puis renvoyer uniquement le texte
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())


CPU times: user 5.9 s, sys: 20.4 ms, total: 5.92 s
Wall time: 5.93 s


BeautifulSoup nous permet de supprimer efficacement la plupart du code html mais pas tout.

In [163]:
df_questions['Body'][10]

'Question: How can I use paste without 100000 becoming 1e+05?\nSorry in advance if this question seems frivolous (but it has resulted in a bug in my code).  I use R to call an external script, so when I say e.g. paste("abc",100000) I want it to output "abc 100000" and not "abc 1e+05".\nHere\'s an example of what it looks like on my screen:\n> paste("abc",100000)\n[1] "abc 1e+05"\n> paste("abc",100001)\n[1] "abc 100001"\n\nThis results in the bizarre behaviour that my script works for the input "100001" but not "100000".\nI realise I could create a script to convert numbers to strings however I like, but I feel I shouldn\'t do this if there is an internal way to do the same thing (I suspect there is some "method" I\'m missing).\n[If it helps, I\'m on Ubuntu 12.04.1 LTS ("precise"), running R version 2.14.1 (2011-12-22) in a terminal.]\n'

Nous devons donc supprimer le reste ici.

In [164]:
def clean_text(text):
    """ mettre la déf de la fonction"""
    text = re.sub(r"\'", "'", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r"\n", " ", text) # match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = re.sub(r"\xa0", " ", text) # match all literal non-breakable space pattern then replace them by a single whitespace
    text = re.sub('\s+', ' ', text) # match all one or more whitespace then replace them by a single whitespace
    text = text.strip(' ')
    return text


In [165]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: clean_text(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: clean_text(x))

CPU times: user 596 ms, sys: 0 ns, total: 596 ms
Wall time: 595 ms


In [166]:
df_questions['Body'][10]

'Question: How can I use paste without 100000 becoming 1e+05? Sorry in advance if this question seems frivolous (but it has resulted in a bug in my code). I use R to call an external script, so when I say e.g. paste("abc",100000) I want it to output "abc 100000" and not "abc 1e+05". Here\'s an example of what it looks like on my screen: > paste("abc",100000) [1] "abc 1e+05" > paste("abc",100001) [1] "abc 100001" This results in the bizarre behaviour that my script works for the input "100001" but not "100000". I realise I could create a script to convert numbers to strings however I like, but I feel I shouldn\'t do this if there is an internal way to do the same thing (I suspect there is some "method" I\'m missing). [If it helps, I\'m on Ubuntu 12.04.1 LTS ("precise"), running R version 2.14.1 (2011-12-22) in a terminal.]'

Nous devons aussi traiter la colonne des tags


## 3.2. Suppression des contractions

In [167]:
def expand_contractions(text):
    """développer les mots raccourcis, e.g. 'don't' to 'do not'"""
    text = contractions.fix(text)
    return text

In [168]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: expand_contractions(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: expand_contractions(x))

CPU times: user 722 ms, sys: 3.62 ms, total: 726 ms
Wall time: 726 ms


In [169]:
df_questions['Body'][10]

'Question: How can I use paste without 100000 becoming 1e+05? Sorry in advance if this question seems frivolous (but it has resulted in a bug in my code). I use R to call an external script, so when I say e.g. paste("abc",100000) I want it to output "abc 100000" and not "abc 1e+05". here is an example of what it looks like on my screen: > paste("abc",100000) [1] "abc 1e+05" > paste("abc",100001) [1] "abc 100001" This results in the bizarre behaviour that my script works for the input "100001" but not "100000". I realise I could create a script to convert numbers to strings however I like, but I feel I should not do this if there is an internal way to do the same thing (I suspect there is some "method" I am missing). [If it helps, I am on Ubuntu 12.04.1 LTS ("precise"), running R version 2.14.1 (2011-12-22) in a terminal.]'

## 3.3. La correction orthographique

Pour 1000 entrées cette correction prends 10 minutes

In [170]:
def autocorrect(text):
    words_id = token.tokenize(text)
    words_correct = [spell(i) for i in words_id]
    return ' '.join(map(str, words_correct)) # Return the text untokenize

#df_questions['Title'] = df_questions['Title'].apply(lambda x: autocorrect(x))
#df_questions['Body'] = df_questions['Body'].apply(lambda x: autocorrect(x))

## 3.4. Mettre en minuscule le texte

Je choisis d'abaisser le texte après le paquet de contractions car celui-ci peut remettre des lettres majuscules lors de la suppression des contractions. La mise en minuscule du texte est une étape classique et utile de la suppression du bruit ou de la normalisation du texte car elle réduit le vocabulaire, normalise le texte et ne coûte presque rien.

In [171]:
%%time

df_questions['Title'] = df_questions['Title'].str.lower()
df_questions['Body'] = df_questions['Body'].str.lower()
df_questions['Tags'] = df_questions['Tags'].str.lower()

CPU times: user 18.9 ms, sys: 3.9 ms, total: 22.8 ms
Wall time: 22.5 ms


In [172]:
df_questions['Body'][10]

'question: how can i use paste without 100000 becoming 1e+05? sorry in advance if this question seems frivolous (but it has resulted in a bug in my code). i use r to call an external script, so when i say e.g. paste("abc",100000) i want it to output "abc 100000" and not "abc 1e+05". here is an example of what it looks like on my screen: > paste("abc",100000) [1] "abc 1e+05" > paste("abc",100001) [1] "abc 100001" this results in the bizarre behaviour that my script works for the input "100001" but not "100000". i realise i could create a script to convert numbers to strings however i like, but i feel i should not do this if there is an internal way to do the same thing (i suspect there is some "method" i am missing). [if it helps, i am on ubuntu 12.04.1 lts ("precise"), running r version 2.14.1 (2011-12-22) in a terminal.]'

# 4) Suppression des caractères

## 4.1. Suppression de la ponctuation, des caractères spéciaux et des nombres

TOUS les caractères non alphabétiques ont été supprimés (y compris la ponctuation, les nombres et les caractères spéciaux). Ainsi, je ne considère pas les mots importants qui peuvent contenir des caractères spéciaux (comme "C #" en programmation).

In [173]:
def remove_punctuation_and_number(text):
    """remove all punctuation and number"""
    return text.translate(str.maketrans(" ", " ", charac)) 

def remove_non_alphabetical_character(text):
    """remove all non-alphabetical character"""
    text = re.sub("[^a-z]+", " ", text) # remove all non-alphabetical character
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    return text

In [174]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_punctuation_and_number(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_punctuation_and_number(x))

CPU times: user 147 ms, sys: 0 ns, total: 147 ms
Wall time: 147 ms


In [175]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_non_alphabetical_character(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_non_alphabetical_character(x))

CPU times: user 878 ms, sys: 0 ns, total: 878 ms
Wall time: 878 ms


In [176]:
df_questions['Body'][10]

'question how can i use paste without becoming e sorry in advance if this question seems frivolous but it has resulted in a bug in my code i use r to call an external script so when i say eg pasteabc i want it to output abc and not abc e here is an example of what it looks like on my screen pasteabc abc e pasteabc abc this results in the bizarre behaviour that my script works for the input but not i realise i could create a script to convert numbers to strings however i like but i feel i should not do this if there is an internal way to do the same thing i suspect there is some method i am missing if it helps i am on ubuntu lts precise running r version in a terminal'

## 4.2. Suppression de la présence d'un seul caractère


Je choisis de supprimer un seul caractère car lorsque nous faisons de la programmation, nous utilisons souvent un seul caractère alphabétique comme nom de variable ("x", "y", "z", etc.). Et j'ai observé que lorsque j'ai essayé de détecter des sujets sans les supprimer, j'ai trouvé beaucoup de sujets avec eux! Et même un sujet que je pourrais nommer "Nom de variable" ...

In [177]:
def remove_single_letter(text):
    """remove single alphabetical character"""
    text = re.sub(r"\b\w\b", "", text) # remove all single letter
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    text = text.strip(" ")
    return text

In [178]:
%%time

#df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_single_letter(x))
#df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_single_letter(x))

# nous ne pouvons pas supprimer les single letters car nous voulons garder la lettre R !!

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


In [179]:
df_questions['Body'][10]

'question how can i use paste without becoming e sorry in advance if this question seems frivolous but it has resulted in a bug in my code i use r to call an external script so when i say eg pasteabc i want it to output abc and not abc e here is an example of what it looks like on my screen pasteabc abc e pasteabc abc this results in the bizarre behaviour that my script works for the input but not i realise i could create a script to convert numbers to strings however i like but i feel i should not do this if there is an internal way to do the same thing i suspect there is some method i am missing if it helps i am on ubuntu lts precise running r version in a terminal'

# 5) Suppression des stopwords

## 5.1. Removing most frequent words

Supprimer les mots les plus fréquents est une étape classique de la NLP. Les mots les plus fréquents n'ajoutent pas beaucoup d'informations dans la plupart des cas (puisqu'ils sont dans presque toutes les phrases). En les supprimant, vous créez plus d'"espace" pour les autres mots qui peuvent avoir des informations plus utiles.
Vous pouvez utiliser des listes prédéfinies à partir de bibliothèques telles que SciKit-Learn, NLTK et autres. Mais sachez que ces listes peuvent être plus problématiques qu'utiles (en particulier la liste scikit-learn, voir [Stop Word Lists in Free Open-source Software Packages](https://www.aclweb.org/anthology/W18-2502.pdf) pour plus d'informations).

In [180]:
def remove_stopwords(text):
    """remove common words in english by using nltk.corpus's list"""
    words_idx = token.tokenize(text)
    filtered = [i for i in words_idx if not i in stop_words]
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [181]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_stopwords(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_stopwords(x))

CPU times: user 2.21 s, sys: 0 ns, total: 2.21 s
Wall time: 2.21 s


In [182]:
df_questions['Body'][10]

'question use paste without becoming e sorry advance question seems frivolous resulted bug code use r call external script say eg pasteabc want output abc abc e example looks like screen pasteabc abc e pasteabc abc results bizarre behaviour script works input realise could create script convert numbers strings however like feel internal way thing suspect method missing helps ubuntu lts precise running r version terminal'

## 5.2. Suppression d'adjectives

Je choisis de supprimer les adjectifs en plus de la liste NLTK. Pourquoi ? Tout simplement parce que lorsque j'ai d'abord essayé de faire une détection de sujet dans un cahier suivant celui-ci et cela améliore ma détection de sujet. Je pensais aussi que les adjectifs n'ajouteraient aucune information utile. En même temps, je pourrais aussi supprimer des verbes avec le même raisonnement. Mais je ne l'ai pas fait parce que l'ensemble de données StackOverflow concerne la programmation. Et en programmation, nous avons beaucoup de verbes, ou de mots qui peuvent être interprétés comme un verbe, qui peuvent être importants ("return", "get", "request", "replace", etc.).

In [183]:

# noinspection PyTypeChecker
def remove_by_tag(text, undesired_tag):
    """remove all words by using ntk tag (adjectives, verbs, etc.)"""
    words_idx = token.tokenize(text) # Tokenize each words
    words_tagged = nltk.pos_tag(tokens=words_idx, tagset=None, lang='eng') # Tag each words and return a list of tuples (e.g. ("have", "VB"))
    filtered = [i[0] for i in words_tagged if i[1] not in undesired_tag] # Select all words that don't have the undesired tags
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [184]:
%%time
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))

CPU times: user 40.9 s, sys: 133 ms, total: 41.1 s
Wall time: 41.1 s


In [185]:
df_questions['Body'][10]

'question use paste without becoming e advance question seems resulted bug code use r call script say pasteabc want output abc abc example looks like pasteabc abc e pasteabc abc results script works input realise could create numbers strings however like feel way thing method missing helps lts precise running r version terminal'

# 6) Stemming / Lemmatisation

Le Stemming et la Lemmatisation sont des opérations qui :
- peuvent améliorer votre temps de calcul en réduisant votre vocabulaire
- aider à généraliser plus facilement en regroupant les mots (ex: "suis", "sont", "être", etc. seront transformés en "être" pour la lemmatisation)

## 6.1. Stemming

Je n'ai pas choisi d'utiliser le stemming ici, mais l'on doit toujours envisager cette alternative car elle est beaucoup moins coûteuse.

Le stemming est le processus de réduction des mots fléchis à leur racine mot, base ou forme de racine - généralement une forme de mot écrit ("fishing", "fished", and "fisher" à la racine "fish"). Il fonctionne généralement en supprimant l'affixe d'un mot. Un affixe peut être un suffixe ou un préfixe (par exemple «-ed», «-ing», etc.). C'est simple mais ne fonctionnera pas lorsque le mot est "irrégulier" ("ran" et "run"). C'est une opération plus simple que la lemmatisation, qui peut suffire dans certains cas, mais peut faire trop d'erreurs dans d'autres cas.

In [186]:
words = ["program", "programs", "programer", "programing", "programers"]
  
for w in words:
    print(w, " : ", stemmer.stem(w))

program  :  program
programs  :  program
programer  :  program
programing  :  program
programers  :  program


In [187]:
def stem_text(text):
    """Stem the text"""
    words_idx = nltk.word_tokenize(text) # tokenize the text then return a list of tuple (token, nltk_tag)
    stemmed_text = []
    for word in words_idx:
        stemmed_text.append(stemmer.stem(word)) # Stem each words
    return " ".join(stemmed_text) # Return the text untokenize

In [188]:
# %%time

# df_questions['Title'] = df_questions['Title'].apply(lambda x: stem_text(x))
# df_questions['Body'] = df_questions['Body'].apply(lambda x: stem_text(x))

## 6.2. Lemmatisation

Comme dit au début, la lemmatisation est le processus de remplacement d'un mot par son lemma (forme canonique ou forme dictionnaire). Mais dans certains cas, un lemmatiseur peut ne pas être en mesure de trouver la bonne racine si vous ne précisez pas le type de mot comme vous pouvez le voir ci-dessous.

In [189]:
print(lemmatizer.lemmatize("stripes", "v"))
print(lemmatizer.lemmatize("stripes", "n"))  
print(lemmatizer.lemmatize("are"))
print(lemmatizer.lemmatize("are", "v"))

strip
stripe
are
be


Une façon de contourner ce problème consiste à utiliser un marqueur et à passer le type de mot dans la fonction lemmatise. MAIS c'est vraiment coûteux. La mise en tige ou une simple lemmatisation à cet égard est bien plus efficace.

In [190]:
def lemmatize_text(text):
    """Lemmatize the text by using tag """
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))  # tokenize the text then return a list of tuple (token, nltk_tag)
    lemmatise_text = []
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'a')) # Lemmatise adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'v')) # Lemmatise verbs
        elif tag.startswith('N'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'n')) # Lemmatise nouns
        elif tag.startswith('R'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'r')) # Lemmatise adverbs
        else:
            lemmatise_text.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatisation
    return " ".join(lemmatise_text) # Return the text untokenize

In [191]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: lemmatize_text(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: lemmatize_text(x))

CPU times: user 39 s, sys: 149 ms, total: 39.2 s
Wall time: 39.4 s


In [192]:
df_questions['Body'][10]

'question use paste without become e advance question seem result bug code use r call script say pasteabc want output abc abc example look like pasteabc abc e pasteabc abc result script work input realise could create number string however like feel way thing method miss help lts precise run r version terminal'

# 7) Feature engineering

L'utilisation du titre et du corps en même temps donne de bien meilleurs résultats pour la détection des sujets.

In [193]:
df_questions['Text'] = df_questions['Title'] + ' ' + df_questions['Body']


# 8) Dataframe avec les Tags

In [194]:
df_questions['Tags'][10]

'<r><r-faq>'

In [195]:
def clean_split_tag(text):
    """ mettre la déf de la fonction"""
    text = re.sub(r"<", "", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r">", ",", text) # match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = text.strip(',')
    text = text.split(',')
    #text = re.sub(r",", " ", text)
    #text = text.rstrip('>')
    return text

In [196]:
%%time

df_questions['Tags'] = df_questions['Tags'].apply(lambda x: clean_split_tag(x))

CPU times: user 135 ms, sys: 36 ms, total: 171 ms
Wall time: 303 ms


In [197]:
tags_list = df_questions[['Id','Tags']].explode('Tags')
print(tags_list.head(10))

def untokenize(words_id):
    return ' '.join(map(str, words_id)) # Return the text untokenize

df_questions['Tags'] = df_questions['Tags'].apply(lambda x: untokenize(x))

         Id             Tags
0   7033844             java
0   7033844           python
0   7033844          node.js
1  13960657           python
1  13960657  lazy-evaluation
2  16632568           python
2  16632568            numpy
3    285061           python
3    285061       attributes
3    285061           object


# 9) Exportation des données

In [198]:
print(df_questions.head(10),'\n \n',tags_list.head(10))
#df_questions.drop(['Tags'], axis=1).to_csv('df_questions_fullclean.csv', encoding='utf-8')
df_questions.to_csv('df_questions_fullclean.csv', encoding='utf-8')
tags_list.to_csv('tags_list.csv', encoding='utf-8')

         Id                                  Title  \
0   7033844        write complicate business logic   
1  13960657       python evaluate condition lazily   
2  16632568                    remove column numpy   
3    285061         programmatically set attribute   
4  16729574               get value cell dataframe   
5  13910357           use request pass item python   
6  34636461       collapse run number string range   
7  11270001  ggplot avoid box around legend symbol   
8     42581   python resub flag replace occurrence   
9   1496346                       pass list kwargs   

                                                Body  \
0  recently get cool package like express jade qu...   
1  example follow statement foo python check cond...   
2  nparray array delete column nphstacknpdeletear...   
3  python object string set x x myattr magic go g...   
4  construct condition extract exactly one row da...   
5  item object need pas along page store data ite...   
6  let u say 